In [1]:
import matplotlib.pyplot as plt
import multiprocessing as mp
import numpy as np

import DissimilarityMetrics
import ChannelChartingCore
import Evaluation
import logging
import optuna

2024-10-10 13:57:34.650959: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-10 13:57:35.463450: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def objective_process(BATCH_SIZE, LEARNING_RATE_INITIAL, LEARNING_RATE_FINAL, MIN_PATHHOPS, MAX_PATHHOPS, TRAINING_BATCHES, ACCELERATION_MEAN, ACCELERATION_VARIANCE, ACCELERATION_WEIGHT, ADP_METRIC_THRESHOLD, ADP_METRIC_SLOPE, ADP_METRIC_OFFSET, ADP_METRIC_VARIANCE_RATIO, REALIZATIONS_COUNT, VELOCITY_VARIANCE, result_queue):
    import dichasus_cf0x
    
    def mean_variance_from_delta_adp(delta_adp):
        mean = np.where(delta_adp == 0, 0, np.where(delta_adp > ADP_METRIC_THRESHOLD, np.inf, delta_adp * ADP_METRIC_SLOPE + ADP_METRIC_OFFSET))
        variance = np.where(delta_adp == 0, 0, np.where(delta_adp > ADP_METRIC_THRESHOLD, np.inf, ADP_METRIC_VARIANCE_RATIO * mean))
    
        return mean, variance
    
    adp_metric = DissimilarityMetrics.ADPDissimilarityMetric(dichasus_cf0x.csi_time_domain, mean_variance_from_delta_adp)

    velocity_mean = 0.25
    velocity_variance = VELOCITY_VARIANCE
    
    # Make worste-case assumption of perfectly correlated velocities. This maximizes the variance.
    velocity_metric = DissimilarityMetrics.VelocityDissimilarityMetric(velocity_mean, velocity_variance, True, dichasus_cf0x.timestamps)

    GDM = ChannelChartingCore.GaussianDissimilarityModel([adp_metric, velocity_metric])
    GDM.generate_short_paths(realization_count = REALIZATIONS_COUNT)

    cc = ChannelChartingCore.ChannelChart(GDM, dichasus_cf0x.csi_time_domain, dichasus_cf0x.timestamps, batch_size = BATCH_SIZE, learning_rate_initial = LEARNING_RATE_INITIAL, learning_rate_final = LEARNING_RATE_FINAL, min_pathhops = MIN_PATHHOPS, max_pathhops = MAX_PATHHOPS, training_batches = TRAINING_BATCHES, acceleration_mean = ACCELERATION_MEAN, acceleration_variance = ACCELERATION_VARIANCE, acceleration_weight = ACCELERATION_WEIGHT)

    channel_chart_positions = cc.predict(dichasus_cf0x.csi_time_domain)

    channel_chart_positions_transformed, errorvectors, errors, mae, cep = Evaluation.mean_absolute_error_transformed(dichasus_cf0x.groundtruth_positions, channel_chart_positions)
    result_queue.put(mae)

In [3]:
def objective(trial):
    # default values:
    #BATCH_SIZE = 3000
    #LEARNING_RATE_INITIAL = 1e-2
    #LEARNING_RATE_FINAL = 1e-2
    #MIN_PATHHOPS = 1
    #MAX_PATHHOPS = 30
    #TRAINING_BATCHES = 2000
    #ACCELERATION_MEAN = 0.8
    #ACCELERATION_VARIANCE = 1.7
    #ACCELERATION_WEIGHT = 0.01
    #ADP_METRIC_THRESHOLD = 20
    #ADP_METRIC_SLOPE = 0.0025
    #ADP_METRIC_OFFSET = 0.004
    #ADP_METRIC_VARIANCE_RATIO = 0.01
    #REALIZATIONS_COUNT = 8
    #VELOCITY_VARIANCE = 0.008

    BATCH_SIZE = trial.suggest_int("BATCH_SIZE", 2000, 4000)
    LEARNING_RATE_INITIAL = trial.suggest_float("LEARNING_RATE_INITIAL", 8e-3, 5e-2)
    LEARNING_RATE_FINAL = trial.suggest_float("LEARNING_RATE_FINAL", 5e-4, 8e-3)
    MIN_PATHHOPS = trial.suggest_int("MIN_PATHHOPS", 1, 8)
    MAX_PATHHOPS = trial.suggest_int("MAX_PATHHOPS", 8, 50)
    TRAINING_BATCHES = trial.suggest_int("TRAINING_BATCHES", 1000, 3000)
    ACCELERATION_MEAN = trial.suggest_float("ACCELERATION_MEAN", 0, 1.4)
    ACCELERATION_VARIANCE = trial.suggest_float("ACCELERATION_VARIANCE", 1.0, 3.0)
    ACCELERATION_WEIGHT = trial.suggest_float("ACCELERATION_WEIGHT", 0.00, 0.03)
    ADP_METRIC_THRESHOLD = trial.suggest_int("ADP_METRIC_THRESHOLD", 15, 23)
    ADP_METRIC_SLOPE = trial.suggest_float("ADP_METRIC_SLOPE", 0.0015, 0.0035)
    ADP_METRIC_OFFSET = trial.suggest_float("ADP_METRIC_OFFSET", 0.002, 0.008)
    ADP_METRIC_VARIANCE_RATIO =  trial.suggest_float("ADP_METRIC_VARIANCE_RATIO", 0.005, 0.02)
    VELOCITY_VARIANCE = trial.suggest_float("VELOCITY_VARIANCE", 0.004, 0.02)
    REALIZATIONS_COUNT = trial.suggest_categorical("REALIZATIONS_COUNT", [8, 16])

    result_queue = mp.Queue()

    p = mp.Process(target=objective_process, args=(BATCH_SIZE, LEARNING_RATE_INITIAL, LEARNING_RATE_FINAL, MIN_PATHHOPS, MAX_PATHHOPS, TRAINING_BATCHES, ACCELERATION_MEAN, ACCELERATION_VARIANCE, ACCELERATION_WEIGHT, ADP_METRIC_THRESHOLD, ADP_METRIC_SLOPE, ADP_METRIC_OFFSET, ADP_METRIC_VARIANCE_RATIO, REALIZATIONS_COUNT, VELOCITY_VARIANCE, result_queue))
    p.start()
    p.join()

    return result_queue.get()

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.FileHandler("official_optuna_log.log", mode="w"))
optuna.logging.enable_propagation()

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1000, gc_after_trial=True)
print(f"Best params is {study.best_params} with value {study.best_value}")

[I 2024-10-10 13:57:36,853] A new study created in memory with name: no-name-d87ae91e-8405-4dda-be1f-ef7824083ba3
2024-10-10 13:57:36.933252: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-10 13:57:36.988017: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-10 13:57:36.988415: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUM

Generating dissimilarity realizations...


  0%|          | 0/2 [00:00<?, ?it/s]

Choosing smallest dissimilarity realization pair-wise...
Running shortest path algorithm...


Computing Shortest Paths:   0%|          | 0/52127500 [00:00<?, ?it/s]

...

Longest short path has 504 hops


I0000 00:00:1728569211.027654  128641 service.cc:145] XLA service 0x7f8e78134b30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728569211.027729  128641 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4080, Compute Capability 8.9
2024-10-10 14:06:51.170012: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-10 14:06:51.502457: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
2024-10-10 14:06:52.292680: W external/local_xla/xla/service/gpu/nvptx_compiler.cc:742] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.3.107). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility

   1/1031 ━━━━━━━━━━━━━━━━━━━━ 1:30:01 5s/step - loss: 474.4917

I0000 00:00:1728569213.634852  128641 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 177/1031 ━━━━━━━━━━━━━━━━━━━━ 2:31 177ms/step - loss: 31.8812